<a href="https://colab.research.google.com/github/genaiconference/Agentic_RAG_Workshop/blob/main/04_Traditional_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Traditional RAG

In [1]:
!git clone https://github.com/genaiconference/Agentic_RAG_Workshop.git

Cloning into 'Agentic_RAG_Workshop'...
remote: Enumerating objects: 206, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 206 (delta 63), reused 36 (delta 36), pack-reused 115 (from 1)
Receiving objects: 100% (206/206), 21.02 MiB | 9.12 MiB/s, done.
Resolving deltas: 100% (96/96), done.


## Setup and Installations
Install necessary libraries for document processing, data handling, and interacting with Azure Document Intelligence and OpenAI.

In [2]:
!pip install -r /content/Agentic_RAG_Workshop/requirements.txt

## Load Environment Variables and Initialize Clients
Load environment variables containing API keys and endpoint information, and initialize the Azure Document Intelligence and OpenAI clients.

In [1]:
import os

os.chdir("/content/Agentic_RAG_Workshop/")

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

load_dotenv()

llm = ChatOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4.1",
    temperature=0,
)

embeddings = OpenAIEmbeddings(
    api_key=os.getenv("OPENAI_API_KEY"),
    model="text-embedding-3-small"
)

## Create Retrievers for all data sources

Use the Retriever Pipeline which does the following:

- Load DI Output
- Generate Parent docs
- Create Child docs
- Generate Summaries
- Generate Question docs
- Create Chroma DB
- Create Ensemble Retriever

In [2]:
from multivector_utils import create_retriever_pipeline

### Retriever for Insurance Policy Document

In [4]:
insurance_policy_retriever = create_retriever_pipeline(
    di_results_filename="Insurance_Policy_results.pkl",
    source_file_name="Insurance_Policy",
    vector_db_name="chroma-insurance-policy",
    embeddings_model=embeddings,
    llm_model=llm,
    vectorstore_exists=True
)

### Retriever for Leave Policy Document

In [5]:
leave_policy_retriever = create_retriever_pipeline(
    di_results_filename="Leave_Policy_results.pkl",
    source_file_name="Leave_Policy",
    vector_db_name="chroma-leave-policy",
    embeddings_model=embeddings,
    llm_model=llm,
    vectorstore_exists=True
)

### Retriever for Microsoft 2023 & 2024 Documents

In [6]:
MS23_retriever = create_retriever_pipeline(
    di_results_filename="Microsoft_2023_results.pkl",
    source_file_name="Microsoft_2023",
    vector_db_name="chroma-microsoft-2023",
    embeddings_model=embeddings,
    llm_model=llm,
    vectorstore_exists=True
)

In [7]:
MS24_retriever = create_retriever_pipeline(
    di_results_filename="Microsoft_2024_results.pkl",
    source_file_name="Microsoft_2024",
    vector_db_name="chroma-microsoft-2024",
    embeddings_model=embeddings,
    llm_model=llm,
    vectorstore_exists=True
)

### Retriever for Apple 2023 & 2024 Documents

In [8]:
AP23_retriever = create_retriever_pipeline(
    di_results_filename="Apple_2023_results.pkl",
    source_file_name="Apple_2023",
    vector_db_name="chroma_apple-2023",
    embeddings_model=embeddings,
    llm_model=llm,
    vectorstore_exists=True
)

In [9]:
AP24_retriever = create_retriever_pipeline(
    di_results_filename="Apple_2024_results.pkl",
    source_file_name="Apple_2024",
    vector_db_name="chroma_apple-2024",
    embeddings_model=embeddings,
    llm_model=llm,
    vectorstore_exists=True
)

## Traditional RAG

add a picture

### Define Prompt

In [10]:
from langchain.prompts import ChatPromptTemplate

template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.
Avoid using generic phrases like "Provide context" or "as per context.

Question: {input}

Context: {context}

Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

## Combine all Retrievers using LOTR - MergerRetriever

**Lord of the Retrievers (LOTR)**, also known as **MergerRetriever**, takes a list of retrievers as input and merges the results of their get_relevant_documents() methods into a single list. The merged results will be a list of documents that are relevant to the query and that have been ranked by the different retrievers.

The MergerRetriever class can be used to improve the accuracy of document retrieval in a number of ways. First, it can combine the results of multiple retrievers, which can help to reduce the risk of bias in the results. Second, it can rank the results of the different retrievers, which can help to ensure that the most relevant documents are returned first.

In [11]:
from langchain.retrievers import MergerRetriever

# The Lord of the Retrievers will hold the output of all retrievers and can be used as any other retriever on different types of chains.
lotr = MergerRetriever(retrievers=[insurance_policy_retriever,
                                   leave_policy_retriever,
                                   MS23_retriever,
                                   MS24_retriever,
                                   AP23_retriever,
                                   AP24_retriever])

### Create Simple RAG Chain

In [12]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# Create a chain to combine documents
combine_docs_chain = create_stuff_documents_chain(llm, prompt)

# Create a retrieval chain
rag_chain = create_retrieval_chain(lotr, combine_docs_chain)

### Test Q&A

In [18]:
from IPython.display import Markdown

question = "I am planning to adopt a kid. How many leaves can I avail and What are the Insurance benefits I can get?"
response = rag_chain.invoke({"input": question})

display(Markdown(response["answer"]))

# Look at the retrieved documents
lotr.invoke(question)

For adoption, you are eligible for 26 weeks of Parental Leave, which must be availed within one year of the child's adoption. Regarding insurance benefits, your group medical policy covers standard hospitalization, pre-existing diseases, maternity (including adoption-related newborn cover from day one with HR intimation within 15 days), and other health benefits for you and your dependents. You can also enroll your adopted child as a dependent under the health insurance plan by notifying HR within 15 days of adoption.

[Document(metadata={'Header 1': 'Employee Benefits Manual 2023-24 Novartis Group', 'Header 2': 'Making A Claim Group Medical- Important FAQ', 'page_number': 35, 'custom_metadata': 'Making A Claim Group Medical- Important FAQ', 'source': 'Insurance_Policy'}, page_content='##Making A Claim Group Medical- Important FAQ\n\n What are network hospitals? What should I do when I reach the hospital (NETWORK)?  \n. These are hospitals where TPA has a tie up for the cashless hospitalization. There are two kinds of network hospitals; PPN Network\nhospitals where cashless services can be obtained for emergency and planned treatments and Standard (Non PPN) network\nhospitals where cashless services can be obtained for planned Hospitalization.  \n. Once you have reached there, please show your ID card for identification. TPA will also send $a$ letter of credit (on pre-authorization)\nto the hospital to make sure that they extend credit facility. Please complete the pre-authorization procedure listed 

In [19]:
question = "Compare the profits of Microsoft & Apple in 2024"
response = rag_chain.invoke({"input": question})

display(Markdown(response["answer"]))

# Look at the retrieved documents
lotr.invoke(question)

In 2024, Microsoft reported a net income (profit) of $89.3 billion for the fiscal year ended June 30, 2024. Apple reported a net income of $46.9 billion for the fiscal year ended September 28, 2024, which was significantly reduced by a one-time $10.2 billion tax charge related to the European Commission State Aid Decision. Therefore, Microsoft's profit in 2024 was substantially higher than Apple's.

[Document(metadata={'Header 1': 'Employee Benefits Manual 2023-24 Novartis Group', 'Header 2': 'Make The Best Use Of Your Benefits', 'page_number': 3, 'custom_metadata': 'Make The Best Use Of Your Benefits', 'source': 'Insurance_Policy'}, page_content='##Make The Best Use Of Your Benefits\n\n <figure>  \nThe process begins with enrollment, which involves planning the enrollment and following the procedure to obtain an E-Card. After this, it is necessary to ensure that all dependent details are accurately listed. This includes providing the name, date of birth, gender, age, and relationship details for each dependent. All this information must be correctly entered into the HR CORE system, which is the Novartis HR Database. This ensures that the enrollment process is complete and all necessary data is captured for both the individual and their dependents.\nGetting Enrolled  \n· Plan Enrollment &\nProcess for getting\nyour E-Card  \nO  \nPlease ensure that all your\ndependent details are 